In [ ]:
# --- Importaciones ---
import os
import time
import requests
import google.generativeai as genai

from bs4 import BeautifulSoup
from dotenv import load_dotenv

# --- Constantes de configuración ---
MAX_CONTENT_LENGTH = 8000
TIMEOUT_SECONDS = 10
MAX_RETRIES = 3


# --- 1. Funciones de extracción ---

def get_page_content(url: str) -> str:
    """Extrae el texto de una URL eliminando contenido irrelevante."""
    try:
        response = requests.get(url, timeout=TIMEOUT_SECONDS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Eliminar elementos no deseados
        for element in soup(["script", "style", "header", "footer", "nav", "aside"]):
            element.decompose()

        return soup.get_text(separator=" ", strip=True)

    except requests.exceptions.RequestException as e:
        return f"Error al acceder a la URL '{url}': {e}"


# --- 2. Funciones de carga de archivos ---

def load_items_from_file(file_path: str) -> list[str]:
    """
    Carga URLs desde un .txt.
    Ignora líneas en blanco y comentarios (#).
    """
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return [
                line.strip()
                for line in file
                if line.strip() and not line.strip().startswith("#")
            ]
    except FileNotFoundError:
        print(f"Advertencia: El archivo '{file_path}' no se encontró. Saltando...")
        return []


def load_prompt_from_file(file_path: str, is_critical: bool = True) -> str | None:
    """Carga un prompt desde archivo. Puede marcarse como crítico o no."""
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return file.read()
    except FileNotFoundError:
        if is_critical:
            print(f"Error: El archivo '{file_path}' no se encontró. No se puede continuar.")
            return None
        print(f"Advertencia: El archivo '{file_path}' no se encontró. Usando contenido vacío.")
        return ""


def load_and_process_content(urls_file: str) -> str:
    """Carga y procesa el contenido de URLs."""
    content_parts = []

    # Procesar URLs
    for url in load_items_from_file(urls_file):
        print(f"Analizando URL: {url}")
        extracted = get_page_content(url)
        if not extracted.startswith("Error"):
            content_parts.append(extracted)
        else:
            print(extracted)

    return "\n".join(content_parts)


# --- 3. Configuración inicial y procesamiento principal ---

def main():
    # Cargar variables de entorno
    load_dotenv()

    # Rutas a archivos de configuración
    prompt_file = "suno_prompt_user.txt"      # Instrucciones del sistema
    urls_file = "suno_urls_a_analizar.txt"         # Analiza las URL's indicadas por el usuario
    user_history_file = "suno_initial_history_user.txt"   # Contexto previo del usuario
    model_history_file = "suno_initial_history_model.txt" # Respuestas previas del modelo

    # Cargar instrucciones del sistema
    system_instruction = load_prompt_from_file(prompt_file, is_critical=True)
    if system_instruction is None:
        return

    # Configurar API de Gemini
    try:
        api_key = os.getenv("GEMINI_API_KEY")
        if not api_key:
            raise ValueError("La clave de API no se encontró en el archivo .env")

        genai.configure(api_key=api_key)
        print("API de Gemini configurada exitosamente.")
    except Exception as e:
        print(f"Error al configurar la API de Gemini: {e}")
        return

    # Cargar y procesar contenido solo desde URLs
    full_content = load_and_process_content(urls_file)
    trimmed_content = full_content[:MAX_CONTENT_LENGTH]

    # Preparar historial inicial
    user_hist = load_prompt_from_file(user_history_file, is_critical=False)
    model_hist = load_prompt_from_file(model_history_file, is_critical=False)

    initial_history = []
    if user_hist:
        initial_history.append({"role": "user", "parts": [user_hist]})

    # Prompt del usuario con contenido analizado
    user_prompt = f"""
{model_hist}

---
CONTENIDO PARA ANÁLISIS:
{trimmed_content}
---

Por favor, confirma que has procesado esta información y que estás listo para recibir mis preguntas.
"""
    initial_history.extend([
        {"role": "user", "parts": [user_prompt]},
        {"role": "model", "parts": ["Información procesada. Estoy listo para generar tus prompts para Suno."]}
    ])

    # Inicializar chat
    model = genai.GenerativeModel("gemini-2.5-pro", system_instruction=system_instruction)
    chat = model.start_chat(history=initial_history)

    # Bucle de interacción
    print("\n\nEstoy listo para generar tu prompt para Suno. Escribe 'salir' para terminar la conversación.")

    while True:
        user_input = input("Tú: ")
        if user_input.lower() == "salir":
            print("Gemini: ¡Hasta la próxima!")
            break

        # Reintentos en caso de error
        for attempt in range(MAX_RETRIES):
            try:
                print("Gemini: Generando tu prompt...")
                response = chat.send_message(user_input)

                if response.text:
                    print("Gemini:", response.text)
                    break
                else:
                    print("\nGemini: Lo siento, no puedo generar una respuesta para esa solicitud debido a las políticas de seguridad.")
                    break

            except Exception as e:
                if "500 An internal error" in str(e) and attempt < MAX_RETRIES - 1:
                    print(f"Error de la API (500). Reintentando en 5 segundos... (Intento {attempt + 1}/{MAX_RETRIES})")
                    time.sleep(5)
                else:
                    print(f"Ocurrió un error inesperado al comunicarse con la API: {e}")
                    break
        else:
            print("Gemini: No se pudo establecer conexión con la API después de varios intentos.")


if __name__ == "__main__":
    main()
